In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv('total_players_data.csv')
df.sample(4)

,Unnamed: 0,player_id,season,team_id,market_value,player_name,birthdate,height,main_position,national_team,foot,age,agent_name,assists,yellow_cards,red_cards,injury,player_app,goals,own_goals
3709,3709,129476,2019,1010,20000000.0,Gerard Deulofeu,1994-03-13,177.0,Centre-Forward,Spain,right,29.0,Unknown,0.0,NaN,NaN,0.0,NaN,NaN,NaN
18577,18577,282810,2019,12,9000000.0,Carles Perez,1998-02-16,173.0,Right Winger,Spain U21,left,25.0,Unknown,0.0,NaN,NaN,0.0,NaN,NaN,NaN
15294,15294,88683,2015,1025,11000000.0,Mattia Destro,1991-03-20,182.0,Centre-Forward,Italy,right,32.0,Unknown,2.0,9.0,0.0,9.0,5.0,8.0,NaN
13021,13021,125614,2019,533,4000000.0,Sargis Adamyan,1993-05-23,184.0,Centre-Forward,Armenia,right,30.0,Karl M. Herzog ...,0.0,NaN,NaN,15.0,15.0,NaN,NaN


In [3]:
df.drop(['Unnamed: 0', 'player_id', 'season', 'team_id', 'player_name', 'birthdate', 'national_team', 'agent_name'], axis=1, inplace=True)
df.head(2)

,market_value,height,main_position,foot,age,assists,yellow_cards,red_cards,injury,player_app,goals,own_goals
0,35000000.0,200.0,Goalkeeper,left,31.0,0.0,1.0,2.0,18.0,26.0,NaN,NaN
1,12000000.0,199.0,Goalkeeper,right,35.0,0.0,NaN,NaN,0.0,22.0,NaN,NaN


In [4]:
med_height = df['height'].median()
med_age = df['age'].median()

In [5]:
df = df.dropna(subset=['market_value'])
df['goals'].fillna(0, inplace=True)
df['own_goals'].fillna(0, inplace=True)
df['yellow_cards'].fillna(0, inplace=True)
df['red_cards'].fillna(0, inplace=True)
df['player_app'].fillna(0, inplace=True)
df['age'].fillna(med_age, inplace=True)
df['foot'].fillna(value='right',inplace=True)
df['height'].fillna(value=med_height, inplace=True)
df['main_position'].fillna(value='unknown', inplace=True)
df.sample(3)

,market_value,height,main_position,foot,age,assists,yellow_cards,red_cards,injury,player_app,goals,own_goals
10754,2750000.0,184.0,Right Midfield,right,33.0,0.0,0.0,0.0,3.0,14.0,0.0,0.0
21203,600000.0,178.0,Defensive Midfield,right,30.0,0.0,1.0,0.0,0.0,6.0,0.0,0.0
10832,3500000.0,193.0,Centre-Forward,left,29.0,0.0,0.0,0.0,7.0,24.0,0.0,0.0


In [6]:
df.isna().sum()

market_value     0
height           0
main_position    0
foot             0
age              0
assists          0
yellow_cards     0
red_cards        0
injury           0
player_app       0
goals            0
own_goals        0
dtype: int64

In [7]:
df['main_position'].value_counts()

unknown               3812
Centre-Back           3620
Centre-Forward        2862
Central Midfield      2650
Goalkeeper            2240
Defensive Midfield    1750
Right-Back            1698
Left-Back             1538
Right Winger          1251
Left Winger           1243
Attacking Midfield    1202
Second Striker         175
Left Midfield          145
Right Midfield         143
Attack                   8
midfield                 3
Defender                 1
Name: main_position, dtype: int64

In [8]:
df = df[df['main_position'] != 'unknown']
df['main_position'].value_counts()

Centre-Back           3620
Centre-Forward        2862
Central Midfield      2650
Goalkeeper            2240
Defensive Midfield    1750
Right-Back            1698
Left-Back             1538
Right Winger          1251
Left Winger           1243
Attacking Midfield    1202
Second Striker         175
Left Midfield          145
Right Midfield         143
Attack                   8
midfield                 3
Defender                 1
Name: main_position, dtype: int64

In [9]:
class_maps = {
    'Forward': ['Centre-Forward', 'Second Striker', 'Attack'], 
    'Midfielder': ['Central Midfield', 'Defensive Midfield', 'Attacking Midfield','Left Midfield', 'Right Midfield', 'midfield'], 
    'Defender': ['Defender', 'Centre-Back','Right-Back', 'Left-Back'], 
    'Winger': ['Right Winger', 'Left Winger'],
    'Goalkeeper': ['Goalkeeper']
}
for key, value in class_maps.items():
    df['main_position'] = df['main_position'].apply(lambda x: key if x in value else x)
df['main_position'].value_counts()

Defender      6857
Midfielder    5893
Forward       3045
Winger        2494
Goalkeeper    2240
Name: main_position, dtype: int64

In [10]:
# def create_dict(lst):
#   return {value:index for index, value in enumerate(lst)}

In [11]:
# df['main_position']=df['main_position'].map(create_dict(df['main_position'].unique()))

In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['main_position'] = le.fit_transform(df['main_position'])
classes_ = list(le.classes_)
print(classes_)
le = LabelEncoder()
df['foot'] = le.fit_transform(df['foot'])

['Defender', 'Forward', 'Goalkeeper', 'Midfielder', 'Winger']


In [13]:
df['main_position'].value_counts()

0    6857
3    5893
1    3045
4    2494
2    2240
Name: main_position, dtype: int64

# Normalization

* because we know maximum of everything in the history of footbal in this task, we can divide all the dataframes by their maximums

In [14]:
df.columns   

Index(['market_value', 'height', 'main_position', 'foot', 'age', 'assists',
       'yellow_cards', 'red_cards', 'injury', 'player_app', 'goals',
       'own_goals'],
      dtype='object')

In [15]:
cols = ['market_value', 'height', 'foot', 'age', 'assists',
       'yellow_cards', 'red_cards', 'injury', 'player_app', 'goals',
       'own_goals']
df[cols] = df[cols].apply(lambda col: col / col.max() if col.name != 'main_position' else col)
df.head(5)

,market_value,height,main_position,foot,age,assists,yellow_cards,red_cards,injury,player_app,goals,own_goals
0,0.17500,0.970874,2,0.333333,0.62,0.0,0.0625,0.666667,0.409091,0.309524,0.000000,0.0
1,0.06000,0.966019,2,0.666667,0.70,0.0,0.0000,0.000000,0.000000,0.261905,0.000000,0.0
2,0.00125,0.922330,2,0.666667,0.82,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.0
3,0.00125,0.966019,2,0.666667,0.58,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.0
4,0.08500,0.922330,0,0.666667,0.56,0.0,0.0625,0.000000,0.340909,0.166667,0.039216,0.0


# Split

In [16]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.1, random_state=46)
print("Number of Train data:", len(train))
print("Number of Test data:", len(test))

Number of Train data: 18476
Number of Test data: 2053


In [17]:
x_train = train.drop('main_position', axis=1)
y_train = train['main_position']
x_test = test.drop('main_position', axis=1)
y_test = test['main_position']

#Apply SVM Grid search

* we first find how much time one iteration will take and set reasonable number of params for the grid search

In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [19]:
import time

t_start = time.time()
clf = SVC(kernel='linear', random_state=42)
clf.fit(x_train, y_train)
print(time.time() - t_start)

26.415889024734497


In [20]:
chosen_arams = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.1, 1, 10, 100],
    # 'kernel': ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'],
    # 'degree':[2, 3, 4]
}

* this will be run 360x5 times and will find the best params

In [21]:
t_start = time.time()
svc = SVC(random_state=42)
# 5 fold cross validation
grid_search = GridSearchCV(svc, chosen_arams, cv=5)

grid_search.fit(x_train, y_train)

print("Best params->", grid_search.best_params_)
print("Cros validation score->", grid_search.best_score_)
print('Elapsed Time->', time.time() - t_start, 'seconds')

Best params-> {'C': 10, 'gamma': 10}
Cros validation score-> 0.49318028047730345
Elapsed Time-> 4762.50182723999 seconds


In [23]:
clf = SVC(C=10, gamma=10, random_state=42)
clf.fit(x_train, y_train)

SVC(C=10, gamma=10, random_state=42)

In [24]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred = clf.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
print('f1 score', f1)
print('accuracy', accuracy)
print('precision', precision)
print('recall', recall)

f1 score 0.4804885611188872
accuracy 0.4929371651242085
precision 0.5077340374080345
recall 0.4929371651242085
